In [1]:
import pandas as pd

In [14]:
df = pd.read_csv('data/devwithoutlabels.txt', sep='\t')

In [15]:
df.head(10)

,id,turn1,turn2,turn3
0,0,Then dont ask me,YOURE A GUY NOT AS IF YOU WOULD UNDERSTAND,IM NOT A GUY FUCK OFF
1,1,Mixed things such as??,the things you do.,Have you seen minions??
2,2,Today I'm very happy,and I'm happy for you ❤,I will be marry
3,3,Woah bring me some,left it there oops,Brb
4,4,it is thooooo,I said soon master.,he is pressuring me
5,5,Wont u ask my age??,hey at least I age well!,Can u tell me how can we get closer??
6,6,I said yes,What if I told you I'm not?,Go to hell
7,7,Where I ll check,why tomorrow?,No I want now
8,8,Shall we meet,you say- you're leaving soon...anywhere you wa...,?
9,9,Let's change the subject,I just did it .l.,You're broken


In [16]:
turn1 = df.turn1.values
turn2 = df.turn2.values
turn3 = df.turn3.values

In [17]:
import os
import pickle

def read_slangdict():
    filename = "slangdict.pickle"
    if os.path.isfile(filename):
        print("Reading data...")
        data = pickle.load(open(filename, 'rb'))
        return data

In [18]:
slang_dict = read_slangdict()

Reading data...


In [19]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
# import json

# with open('emoji/emoji_ranks_data_story.json', 'r') as fn:
#     emoticons = json.load(fn)

def ws_tokenizer(text):
    return text.split()


text_processor = TextPreProcessor(
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user', 'time',
               'date', 'number'],
    annotate={"hashtag", "elongated", "allcaps", "repeated", 'emphasis',
              'censored'},
    all_caps_tag="wrap",
    fix_text=True,
    segmenter="twitter_2018",
    corrector="twitter_2018",
    unpack_hashtags=True,
    unpack_contractions=True,
    spell_correct_elong=True,
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    dicts=[emoticons,slang_dict]
)

Reading twitter_2018 - 1grams ...
Reading twitter_2018 - 2grams ...
Reading twitter_2018 - 1grams ...


In [20]:
norm_turn1, norm_turn2, norm_turn3 = [], [], []

for t1, t2, t3 in zip(turn1, turn2, turn3):
    norm_turn1.append(" ".join(text_processor.pre_process_doc(t1)))
    norm_turn2.append(" ".join(text_processor.pre_process_doc(t2)))
    norm_turn3.append(" ".join(text_processor.pre_process_doc(t3)))

In [21]:
print(norm_turn1[0:20])

["then don't ask me", 'mixed things such as ? <repeated>', 'today i am very happy', 'woah bring me some', 'it is the <elongated>', 'wont u ask my age ? <repeated>', 'i said yes', 'where i ll check', 'shall we meet', 'let us change the subject', 'your picture peace', 'not mine', 'i want to play the game', 'iam sory', 'how much', 'okay', 'so the story ?', 'may be', 'so come on not applicable . <repeated> want u so badly .', 'no you are not']


In [22]:
# label = df.label.values.tolist()
idx = df.id.values.tolist()

In [23]:
# norm_data = {'id': idx, 'turn1': norm_turn1, 'turn2': norm_turn2,
#              'turn3': norm_turn3, 'label': label}
norm_data = {'id': idx, 'turn1': norm_turn1, 'turn2': norm_turn2,
             'turn3': norm_turn3}
df_norm = pd.DataFrame(data=norm_data)

In [24]:
df_norm.head(10)

,id,turn1,turn2,turn3
0,0,then don't ask me,<allcaps> youre a guy not as if you would unde...,<allcaps> Instant Message not a guy fuck off <...
1,1,mixed things such as ? <repeated>,the things you do .,have you seen minions ? <repeated>
2,2,today i am very happy,and i am happy for you <happy>,i will be marry
3,3,woah bring me some,left it there oops,be right back
4,4,it is the <elongated>,i said soon master .,he is pressuring me
5,5,wont u ask my age ? <repeated>,hey at least i age well !,can u tell me how can we get closer ? <repeated>
6,6,i said yes,what if i told you i am not ?,go to hell
7,7,where i ll check,why tomorrow ?,no i want now
8,8,shall we meet,you say - you are leaving soon . <repeated> an...,?
9,9,let us change the subject,i just did it . l .,you are broken


In [25]:
df_norm.to_csv('data/devwithoutlabels_norm_3.csv', index=None, sep='\t')